### Loading Dataset

In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("tree_term_climb.R"))
suppressPackageStartupMessages(library(beaveR))

saveDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts/environment/real_datasets/GSE100505_EDL_MAST_subsamp/mode=gcbias/post_type=gibbs_nreps=100_tf=100/DE"
load(file.path(saveDir, "tseConsUp.RData"))

In [8]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRVUp.RData"))

In [3]:
treeCons <- rowTree(tseConsUp)
l <- length(treeCons$tip)

quantDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST_subsamp/sal_out/mode_gcbias=True/posttype=gibbs_npost=100_tf=100"
metaData <- read.delim("/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST/SRR_Acc_List.txt")
samples <- metaData$RUN
files <- file.path(quantDir, samples, "quant.sf")
colData <- cbind(data.frame(files = files, names = samples), condition = as.factor(metaData$TissueName))
seMuscle <- tximeta::tximeta(colData)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Mus musculus - release M25 ]

loading existing TxDb created: 2023-02-17 18:26:37

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

loading existing transcript ranges created: 2023-02-17 18:26:39

fetching genome info for GENCODE



In [7]:
treeCons <- rowTree(tseConsUp)
l <- length(treeCons$tip)
y <- fishpond::scaleInfReps(tseConsUp[1:l,])
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

In [15]:
txpsSubSamp <- lapply(c(0.01, 0.05, 0.1), function(alpha) rownames(y)[which(mcols(y)[["qvalue"]] < alpha)])
txpsSubDesc <- lapply(tAfterBHMoreClimbMIRVUp[["mIRV=0.4"]], function(nodes) treeCons$tip[unlist(Descendants(treeCons, nodes))])
sapply(txpsSubSamp, length)
sapply(txpsSubDesc, length)

[1] 2751 5539 7692

[1]  1465  9109 12935

In [7]:
yAll <- computeSizeFactors(tseConsUp)
yAll <- scaleInfReps(yAll)
yAll <- labelKeep(yAll)
set.seed(1)
yAll <- swish(yAll, x="condition")
sum(mcols(yAll)[["keep"]])

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

[1] 95860

#### Finding differentially expressed transcripts on the subsampled dataset

In [15]:
dTxpsSub <- lapply(c(0.01,0.05,0.1), function(x) rownames(y)[which(mcols(y)[["qvalue"]] <= x)])
sapply(dTxpsSub, length)

[1] 2753 5540 7696

In [16]:
mapDf <- rowData(seMuscle)
txpGenes <- lapply(dTxpsSub, function(txps) unique(unlist(mapDf[txps, "gene_id"])))##Genes that map to transcripts
sapply(txpGenes, length)

[1] 2398 4529 5961

#### Finding differentially expressed genes on the subsampled dataset

In [19]:
gse <- summarizeToGene(seMuscle)
yg <- fishpond::scaleInfReps(gse)
yg <- labelKeep(yg)
set.seed(1)
yg <- swish(yg, x="condition")

loading existing TxDb created: 2023-02-17 18:26:37

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2023-02-21 05:37:24

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



In [20]:
dgesSub <- lapply(c(0.01,0.05,0.1), function(x) rownames(yg)[which(mcols(yg)[["qvalue"]] <= x)])
sapply(dgesSub, length)

[1] 1933 3553 4890

In [16]:
envMain <- "environment/real_datasets/GSE100505_EDL_MAST/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/DE"
load(file.path(envMain, "dTxps.RData"))
load(file.path(envMain, "dges.RData"))
load(file.path(envMain, "tseConsUp.RData"))
load(file.path(envMain, "tAfterBHMoreClimbMIRVUp.RData"))
treeConsO <- rowTree(tseConsUp)
txpsDesc <- lapply(tAfterBHMoreClimbMIRVUp[["mIRV=0.4"]], function(nodes) treeConsO$tip[unlist(Descendants(treeConsO, nodes))])

In [29]:
sapply(txpsSubDesc, length)
sapply(mapply(function(x,y) intersect(x,y), txpsDesc, txpsSubDesc), length) ##Original Sample treeDE, susample treeDE
sapply(mapply(function(x,y) intersect(x,y), txpsDesc, txpsSubSamp), length) ##Original Sample treeDE, susample DTE
sapply(mapply(function(x,y) intersect(x,y), dTxps, txpsSubDesc), length) ##Original Sample DTE, susample treeDE
sapply(mapply(function(x,y) intersect(x,y), dTxps, txpsSubSamp), length) ##Original Sample DTE, susample DTE

[1]  1465  9109 12935

[1]  1333  7952 11292

[1]  997 4761 6787

[1]  854 4685 7068

[1] 2486 4986 7007

#### TreeDE, with the total number of nodes, inner nodes and their height distribution

In [25]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRVUp.RData"))
treeDESub <- tAfterBHMoreClimbMIRVUp[["mIRV=0.4"]]
sapply(treeDESub, length) ## number of differentially expressed nodes
sapply(treeDESub, function(nodes) sum(nodes > l)) ## number of inner nodes
lapply(treeDESub, function(nodes) table(node.depth(treeCons,2)[nodes])) ## Distribution of node heights

[1]  807 5277 7814

[1]  333 2020 2750

[[1]]

  1   2   3   4   5   6   7   8   9 
474 174  86  36  18  11   2   3   3 

[[2]]

   1    2    3    4    5    6    7    8    9   10   11   12 
3257 1091  530  196  105   55   21   11    7    1    2    1 

[[3]]

   1    2    3    4    5    6    7    8    9   10   11   12   15 
5064 1545  674  267  140   68   24   13   11    2    4    1    1 


In [21]:
### Intersection for transcripts and genes between subsampled and original dataset
commTxps <- sapply(seq_along(dTxps), function(i) intersect(dTxps[[i]], dTxpsSub[[i]]))
sapply(commTxps, length)
diffTxps <- sapply(seq_along(dTxps), function(i) setdiff(dTxps[[i]], dTxpsSub[[i]]))
sapply(diffTxps, length)
                   
commGenes <- sapply(seq_along(dges), function(i) intersect(dges[[i]], dgesSub[[i]]))
sapply(commGenes, length)
diffGenes <- sapply(seq_along(dges), function(i) setdiff(dges[[i]], dgesSub[[i]]))
sapply(diffGenes, length)

[1] 2486 4990 7007

[1]  898 1794 2634

[1]  949 3314 4580

[1]   55 1362 1629

#### Number of differential transcripts in the original data mapped by the treeDE on the subsampled data

In [27]:
txpsTreeDE <- lapply(treeDESub, function(nodes) treeCons$tip[unlist(Descendants(treeCons, nodes))])
sapply(txpsTreeDE, length)
mapply(function(x,y) sum(x %in% y), txpsTreeDE, dTxps)

[1]  1465  9109 12935

[1]  854 4685 7068

#### Number of differentially expressed transcripts for both

In [8]:
sum(mcols(y)[treeCons$tip,"pvalue"] < 0.01, na.rm=T)
sum(mcols(y)[treeCons$tip,"pvalue"] < 0.05, na.rm=T)
sum(mcols(y)[treeCons$tip,"pvalue"] < 0.1, na.rm=T)

sum(mcols(yAll)[treeCons$tip,"pvalue"] < 0.01, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"pvalue"] < 0.05, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"pvalue"] < 0.1, na.rm=T)

[1] 5800

[1] 9747

[1] 12368

[1] 4087

[1] 7976

[1] 10603

In [9]:
sum(mcols(y)[["keep"]])
sum(mcols(yAll)[["keep"]])

[1] 32333

[1] 95860